In [ ]:
# | default_exp cli

In [ ]:
# | export

from typing import *
import os
import re

import typer

from fastkafka_gen._components.logger import get_logger
from fastkafka_gen._code_generator.app_description_validator import validate_app_description
from fastkafka_gen._code_generator.asyncapi_spec_generator import generate_asyncapi_spec
from fastkafka_gen._code_generator.app_generator import generate_app
from fastkafka_gen._code_generator.test_generator import generate_test
from fastkafka_gen._code_generator.helper import set_logger_level, add_tokens_usage
from fastkafka_gen._code_generator.constants import DEFAULT_MODEL, MODEL_PRICING, TOKEN_TYPES

In [ ]:
from typer.testing import CliRunner
import pytest
from unittest.mock import patch

from fastkafka_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

In [ ]:
runner = CliRunner()

In [ ]:
# | export

OPENAI_KEY_EMPTY_ERROR = "Error: OPENAI_API_KEY cannot be empty. Please set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again.\nYou can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details."
OPENAI_KEY_NOT_SET_ERROR = "Error: OPENAI_API_KEY not found in environment variables. Set a valid OpenAI API key in OPENAI_API_KEY environment variable and try again. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details."


def _ensure_openai_api_key_set() -> None:
    """Ensure the 'OPENAI_API_KEY' environment variable is set and is not empty.

    Raises:
        KeyError: If the 'OPENAI_API_KEY' environment variable is not found.
        ValueError: If the 'OPENAI_API_KEY' environment variable is found but its value is empty.
    """
    try:
        openai_api_key = os.environ["OPENAI_API_KEY"]
        if openai_api_key == "":
            raise ValueError(OPENAI_KEY_EMPTY_ERROR)
    except KeyError:
        raise KeyError(OPENAI_KEY_NOT_SET_ERROR)

In [ ]:
with patch.dict(os.environ, {"OPENAI_API_KEY": ""}):
    with pytest.raises(ValueError) as e:
        _ensure_openai_api_key_set()

print(e.value)
assert str(e.value) == OPENAI_KEY_EMPTY_ERROR

In [ ]:
with patch.dict(os.environ, {}, clear=True):
    with pytest.raises(KeyError) as e:
        _ensure_openai_api_key_set()
        
print(e.value)
assert str(e.value) == f"'{OPENAI_KEY_NOT_SET_ERROR}'"

In [ ]:
with patch.dict(os.environ, {"OPENAI_API_KEY": "INVALID_KEY"}):
    _ensure_openai_api_key_set()

In [ ]:
# | export

app = typer.Typer(
    short_help="Commands for accelerating FastKafka app creation using advanced AI technology",
     help="""Commands for accelerating FastKafka app creation using advanced AI technology.

These commands use a combination of OpenAI's gpt-3.5-turbo and gpt-3.5-turbo-16k models to generate FastKafka code. To access this feature, kindly sign up if you haven't already and create an API key with OpenAI. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

Once you have the key, please set it in the OPENAI_API_KEY environment variable before executing the code generation commands.

Note: Accessing OpenAI API incurs charges. However, when you sign up for the first time, you usually get free credits that are more than enough to generate multiple FastKafka apps. For further information on pricing and free credicts, check this link: https://openai.com/pricing
    """,
)

In [ ]:
# | export


def _strip_white_spaces(description: str) -> str:
    """Remove and strip excess whitespaces from a given description

    Args:
        description: The description string to be processed.

    Returns:
        The cleaned description string.
    """
    pattern = re.compile(r"\s+")
    return pattern.sub(" ", description).strip()

In [ ]:
fixture = """
    I have   a                  lot
                of whitespaces
                
                
"""

expected = "I have a lot of whitespaces"
actual = _strip_white_spaces(fixture)
print(actual)
assert actual == expected

In [ ]:
# | export


def _calculate_price(total_tokens_usage: Dict[str, int], model: str = DEFAULT_MODEL) -> float:
    """Calculates the total price based on the number of promt & completion tokens and the models price for input and output tokens (per 1k tokens).

    Args:
        total_tokens_usage: OpenAI "usage" dictionaries which defines prompt_tokens, completion_tokens and total_tokens


    Returns:
        float: The price for used tokens
    """
    model_price = MODEL_PRICING[model]
    price = (total_tokens_usage["prompt_tokens"] * model_price["input"] + total_tokens_usage["completion_tokens"] * model_price["output"]) / 1000
    return price

In [ ]:
usage = {
    "prompt_tokens": 129,
    "completion_tokens": 1,
    "total_tokens": 130
  }

usage_list = [usage, usage]
total_tokens_usage = add_tokens_usage(usage_list)

assert _calculate_price(total_tokens_usage) == 0.000782

In [ ]:
# | export


@app.command(
    "generate",
    help="Effortlessly generate an AsyncAPI specification, FastKafka application code, and integration tests from the app description.",
)
@set_logger_level
def generate_fastkafka_app(
    description: str = typer.Argument(
        ...,
        help="""Summarize your FastKafka app in a few sentences!


\nInclude details about messages, topics, servers, and a brief overview of the intended business logic.


\nThe simpler and more specific the app description is, the better the generated app will be. Please refer to the below example for inspiration:


\nCreate a FastKafka app using localhost broker for testing, staging.example-domain.ai for staging and prod.example-domain.ai for production. Use default port number.

It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.
For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.

Use SASL_SSL with SCRAM-SHA-256 for authentication with username and password.


\n"""
    ),
    output_path: str = typer.Option(
        "./fastkafka-gen",
        "--output_path",
        "-o",
        help="The path to the output directory where the generated files will be saved. This path should be relative to the current working directory.",
    ),
    verbose: bool = typer.Option(
        False,
        "--verbose",
        "-v",
        help="Enable verbose logging by setting the logger level to INFO.",
    ),
) -> None:
    """Effortlessly generate an AsyncAPI specification, FastKafka application code, and integration tests from the app description."""
    try:
        _ensure_openai_api_key_set()
        
        cleaned_description = _strip_white_spaces(description)
        validated_description, description_token = validate_app_description(cleaned_description)

        asyncapi_spec_token = generate_asyncapi_spec(validated_description, output_path)
        app_token = generate_app(output_path)
        test_token = generate_test(validated_description, output_path)
        
        total_tokens_usage = add_tokens_usage([asyncapi_spec_token, app_token, test_token])
        price = _calculate_price(total_tokens_usage)
        
        typer.secho(f" ▶ Total tokens usage: {total_tokens_usage['total_tokens']}", fg=typer.colors.CYAN)
        typer.secho(f" 🤑 Total price: {round(price, 5)}💲", fg=typer.colors.CYAN)
        typer.secho("✨  All files were successfully generated!", fg=typer.colors.CYAN)
    
    except (ValueError, KeyError) as e:
        typer.secho(e, err=True, fg=typer.colors.RED)
        raise typer.Exit(code=1)
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(code=1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(app, ["generate", "--help"])